In [ ]:
# for speechbrain
!pip install -qq torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 torchtext==0.12.0
!pip install -qq speechbrain==0.5.12

# pyannote.audio
!pip install -qq pyannote.audio
!pip install audio2numpy
# for visualization purposes
!pip install -qq moviepy ipython==7.34.0

!pip install pydub
!pip install transformers
!pip install pyctcdecode
!pip install pypi-kenlm

# !pip install asrecognition

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
audio_file = '/content/drive/My Drive/Audio - Speech/Speaker recognition/split_audio lll.wav'
#AUDIO TIENE QUE ESTAR EN FORMATO WAV, EN 16KHZ Y EN MONO. SINO VA A ROMPER

In [ ]:
#PARA ESCUCHARLO

# from pyannote.audio import Audio 
# from IPython.display import Audio as IPythonAudio
# from pyannote.core import Segment, notebook
# EXCERPT = Segment(0, 30)
# waveform, sr = Audio().crop(audio_file, EXCERPT)

In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization') 
diarization = pipeline(audio_file, num_speakers=2)

In [ ]:
diarization

In [ ]:
from pydub import AudioSegment

#SEGMENTO SPEAKERS
segmentos_speaker_0=[]
segmentos_speaker_1=[]
for turn, _, speaker in diarization.itertracks(yield_label=True):
  if speaker=='SPEAKER_00':
    segmentos_speaker_0.append([turn.start*1000,turn.end*1000])
  else:
    segmentos_speaker_1.append([turn.start*1000,turn.end*1000])

a = AudioSegment.from_wav(audio_file)
s_inicial = AudioSegment.silent(0) #tiempo máximo del audio
s1 = AudioSegment.silent(segmentos_speaker_0[-1][1]) #tiempo máximo del audio
contador=0
final_audio_speaker1=s_inicial
aux=0
for i,j in segmentos_speaker_0:
  if aux!=0:
    final_audio_speaker1=final_audio_speaker1+s1[f_anterior:i]+a[i:j]
  else:
    final_audio_speaker1=s1[0:i]+a[i:j]
  f_anterior=j
  aux=aux+1


  # Modifico formato para correrle speech to text

import numpy as np
sample=final_audio_speaker1.get_array_of_samples()
sample = np.array(sample)
sample
sample = sample.astype('float')

In [ ]:
final_audio_speaker1

In [ ]:
# SPEECH TO TEXT

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2FeatureExtractor

modelo="jonatasgrosman/wav2vec2-large-xlsr-53-spanish" #"patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm" #"jonatasgrosman/wav2vec2-large-xlsr-53-spanish"
model = Wav2Vec2ForCTC.from_pretrained(modelo)
processor = Wav2Vec2Processor.from_pretrained(modelo)

In [ ]:
# import audio2numpy as a2n
# audio_file2=(a2n.audio_from_file(audio_file))

In [ ]:
inputs = processor(sample, sampling_rate=16_000, return_tensors="pt")

import torch
with torch.no_grad():
  logits = model(**inputs).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
transcription[0].lower()